In [1]:
def bits2str(bits):
    r = b''
    for i in range(0, len(bits), 8):
        r += int(bits[i:i+8], 2).to_bytes(1, 'big')
    return r.decode('latin_1')

In [2]:
import sounddevice as sd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time
import scipy.io.wavfile as wav
from scipy.fftpack import fft, ifft, fftfreq
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.figsize'] = [16, 5]

In [3]:
from fsk16 import SoundCommunication2, str2bits

In [22]:
msg = """
[ 7071.657145] CPU3: Core temperature above threshold, cpu clock throttled (total events = 88)
[ 7071.657156] CPU2: Core temperature above threshold, cpu cloc
"""
r = str2bits(msg)
FS = 44100
sync = ''.join(map(lambda b: str(int(b)), np.random.rand(400) > 0.5))
comm = SoundCommunication2(FS=FS, msgSymLen=len(r)//4,
                    sync=sync)
from testing import Tester
t = Tester(comm)
S = comm.send(r)
assert comm.decode(S) == r

In [25]:
W = sd.playrec(t.pad(S), samplerate=FS, channels=1, blocking=False).ravel()

In [14]:
WN = t.addBandpassNoise(W, 0, 2)

In [26]:
wr = comm.decode(W)
t.diff(wr, r, chunk_len=4)

('XXXX1010XXXXXXXXXXXXXXXX0011XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX11100011XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX0100XXXXXXXX0101XXXXXXXXXXXX0100XXXX0101XXXX01010101XXXX0011XXXXXXXXXXXXXXXX0100XXXX01101111XXXXXXXX0110XXXXXXXXXXXX0111010001100101XXXX11010111XXXX0110XXXX0111XXXX0110XXXX01110100XXXXXXXXXXXXXXXX01100101XXXXXXXX0110XXXXXXXXXXXXXXXX1111XXXXXXXX0110XXXXXXXXXXXX01110100011010000111XXXX0110XXXXXXXXXXXX01101000XXXX1111XXXX1100XXXX0100XXXXXXXXXXXXXXXX0110XXXXXXXXXXXXXXXXXXXXXXXXXXXX01100011XXXXXXXXXXXX1111XXXX0011XXXXXXXXXXXXXXXX01110100011010000111XXXX01101111XXXX0100XXXX010001101100XXXXXXXXXXXX0100XXXXXXXXXXXXXXXX01110100XXXX1111XXXX0100XXXXXXXXXXXXXXXXXXXXXXXX0110XXXXXXXXXXXX0110XXXXXXXX111001110100XXXXXXXXXXXXXXXX0011XXXXXXXXXXXX0011XXXXXXXX1000XXXX1001XXXX1010XXXXXXXXXXXXXXXX0011XXXXXXXX0000XXXX0111XXXXXXXXXXXX11100011XXXXXXXXXXXXXXXX0111XXXXXXXXXXXX0101XXXX0110XXXXXXXXXXXXXXXX0100XXXX0101XXXXXXXX0101XXXXXXXXXXXX1010XXXXXXXX0100XXXX01101111XXXXXXXX0110XXXXXXXXXXXX0111010001100101XXXX11